<a href="https://colab.research.google.com/github/dhdbsrlw/Korean-Song-Genre-Classification/blob/main/final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Korean Ver Import (local 에 설치 필요)
# koBERT tokenizer용 라이브러리
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install sentencepiece
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf' # koBERT tokenizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 38.3 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292696 sha256=f40883bd450a1d542cb933f7520e0c259c6bf07be1b442884d2557545d46f021
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.5 MB/s eta 0:00:00
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-yts52yz_/kobert-tokenizer_af0d98a1e8d7432198e7f09b5e87c940
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-yts52yz_/kobert-tokenizer_

In [ ]:
!pip install torch
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from os import path
from datetime import datetime

# for koBERT
from kobert_tokenizer import KoBERTTokenizer
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from tqdm import tqdm
import sentencepiece

In [ ]:
import numpy as np
import math

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertForSequenceClassification.from_pretrained('monologg/kobert',num_labels=7)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Import Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load CSV.
#df = pd.read_csv('/content/drive/MyDrive/new_train.csv', header=None, names=['label', 'lyric'])
df = pd.read_csv('/content/drive/MyDrive/D2D/2023_SUMMER_AIKUTHON/new_train.csv', header=None, names=['label', 'lyric'])
df.head()

,,,,,label,lyric
song_name,adults,artist,album_id,date,genre,lyrics
긴 잠,0.0,INO,'1869',2001.03.16,0,잠을 깨는 것이 싫었어\n눈 뜨면 또 하루\n니 곁을 살테니\n오직 내꿈 속엔 넌 ...
Alcatraz,0.0,INO,'1869',2001.03.16,1,이젠 널 가둬놓겠어\n나의 품에\n조금은 낯설겠지만\n편해질꺼야\n두려운 내 맘 때...
해요,0.0,INO,'1869',2001.03.16,0,그녀와 나는요 그땐 참 어렸어요\n많이 사랑했고 때론 많이 다퉜었죠\n지금 생각하면...
투비(鬪悲),0.0,INO,'1869',2001.03.16,0,예전처럼 다시 처음으로\n서로 몰랐던 때로 돌아가\n쉽진 않지만\n부탁이야 잊어줘\...


In [ ]:
# Split datas into frames.

labels = list(map(int,df['label'].tolist()[1:110000]))
lyrics = df['lyric'][1:110000].tolist()

# label_dict = {label: i for i, label in enumerate(set(labels))} # 필요시 삭제
# labels = [label_dict[label] for label in labels]

'''
prev_model_path = 'lyrics_model'

if path.exists(prev_model_path):
    tokenizer = BertTokenizer.from_pretrained(prev_model_path)
    model = BertForSequenceClassification.from_pretrained(prev_model_path)
    print("Model Loaded.", model)
else:
'''

print(labels)
#print(lyrics)

[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 2, 0, 0, 2, 2, 2, 0, 2, 2, 0, 2, 4, 5, 5, 5, 2, 5, 5, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 7, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 0, 2, 0, 0, 2, 0, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 2, 2, 0, 2, 2, 0, 2, 0, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 2, 0, 2, 2, 

In [ ]:
label_dict = {label: i for i, label in enumerate(set(labels))} #

In [ ]:
labels = [label_dict[label] for label in labels] #

In [ ]:
def cls_preprocess(dataset):

  preprocessed = []

  for i in range(len(dataset)):
    dataset[i] = "[CLS]" + dataset[i] + "[SEP]"
    preprocessed.append(dataset[i])

  return preprocessed


In [ ]:
cls_preprocess(lyrics)

['[CLS]잠을 깨는 것이 싫었어\n눈 뜨면 또 하루\n니 곁을 살테니\n오직 내꿈 속엔 넌 오지 않았어\n이것이 나에겐 휴식이었지\n다툼없는 사랑이란걸\n누군가 말했어 다 끝난거라고\n식어간 마음을 서로 잘 알면서\n우리는 왜 헤어지지 못할까\n사랑이란 추억이 아니야\n이건 차라리 이별만도 못해\n얼마든지 우린\n또 사랑할수도 있는데\n이토록 힘든건 사랑이 아닐꺼야\n그래서 난 널 잊기로 했어\n이젠 괜찮아\n우리 사랑이 없던 것처럼\n미움조차 지웠으니\n\n이렇게 나 널 미워하는게\n너무 힘들어 너도 그럴테지\n네 잘못이란거\n날 만났다는거 그 뿐인데\n이토록 힘든건 사랑이 아닐꺼야\n그래서 난 널 잊기로 했어\n이젠 괜찮아\n우리 사랑이 없던 것처럼\n미움조차 지웠으니\n죽도록 난 겁이나\n어느날 널 마주치려는\n모진 우연까지도\n약속해 주겠니\n너를 흔드는 내 소식을 들어도\n날 모른다 할 수 있다고\n부탁이야 넌\n꼭 좋은 사람 만나서\n나 같은 사람 빨리 잊어줘\n나를 용서할 이유도 없는\n니가 되어줘\n나도 그럴테니까\n우린 끝난거니까\n \n[SEP]',
 '[CLS]이젠 널 가둬놓겠어\n나의 품에\n조금은 낯설겠지만\n편해질꺼야\n두려운 내 맘 때문야\n넌 언젠가 날\n떠나버릴 것만 같은\n그런 모습 난 보여줄게 없어\n이렇게 밖에는\n널 잡을 수 없을까봐\n네게 자신할 수 없는\n미래가 불안해\n그래도 너를 포기 못해\nNow I will lock you\nin my heart\n지금 모습 그대로\n내 맘속에 널 영원히 구속할래\nNow make you\nprisoner of my love\n너를 내게 맡겨봐\n지금보다 더 니 모든 걸 가질래\n\n이기적일지도 몰라\n난 너에게는 선택 할\n기회마저도 뺏을 테니\n난 언제나 널 보면\n세상을 다 가진\n그런 느낌이 좋았어\n절대 놓치지 않겠어\n난 죽는다 해도\n그 만큼 너를 포기 못해\nNow I will lock you\nin my heart\n지금 모습 그대로\n내 맘속에 널 영원히 구

In [ ]:
'''
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_dict))
print("New model created.", model)
'''

'\ntokenizer = BertTokenizer.from_pretrained(\'bert-base-uncased\')\nmodel = BertForSequenceClassification.from_pretrained(\'bert-base-uncased\', num_labels=len(label_dict))\nprint("New model created.", model)\n'

In [ ]:
# tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
# model = BertForSequenceClassification.from_pretrained('monologg/kobert')
'''
from kobert_transformers import get_kobert_model
from transformers import BertModel, DistilBertModel
model = get_kobert_model() # BertModel.from_pretrained("monologg/kobert")

from kobert_transformers import get_tokenizer
tokenizer = get_tokenizer()
'''
# tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
# tokenizer.tokenize("[CLS] 한국어 모델을 공유합니다. [SEP]")

'\nfrom kobert_transformers import get_kobert_model\nfrom transformers import BertModel, DistilBertModel\nmodel = get_kobert_model() # BertModel.from_pretrained("monologg/kobert")\n\nfrom kobert_transformers import get_tokenizer\ntokenizer = get_tokenizer()\n'

In [ ]:
# encode lyrics

encoded_lyrics = tokenizer.batch_encode_plus(
    lyrics,
    add_special_tokens=True,
    padding='longest',
    truncation=True,
    max_length=512, # 수정
    return_attention_mask=True,
    return_tensors='pt'
)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
import torch.nn.functional as F

In [ ]:
#one_hot_label_ = F.one_hot(torch.Tensor(labels, num_classes=7)

In [ ]:
# lyrics class

class LyricsDataset(Dataset):
    def __init__(self, lyrics, labels, attention_masks):
        self.lyrics = lyrics
        self.labels = labels
        self.attention_masks = attention_masks
        self.num_classes = 7
        self.one_hot_labels = torch.zeros(len(labels), self.num_classes)
        for i, label in enumerate(self.labels):
              #print(i,label)
              self.one_hot_labels[i, label] = 1.0

    def __len__(self):
        return len(self.lyrics)

    def __getitem__(self, idx):
        return {
            'lyric': self.lyrics[idx],
            'label': self.one_hot_labels[idx],
            'attention_mask': self.attention_masks[idx],
            'gt_label': self.labels[idx]
        }


        num_classes = len(set(labels))
        self.one_hot_labels = torch.zeros(len(labels), num_classes)
        for i, label in enumerate(labels):
            self.one_hot_labels[i, label] = 1.0

# Create dataset

dataset = LyricsDataset(
    lyrics=encoded_lyrics['input_ids'],
    labels=labels,
    attention_masks=encoded_lyrics['attention_mask'],
)


# Split dataset

def dataset_split(dataset, ratio):
    train_size = int(ratio * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
    return train_dataset, val_dataset


In [ ]:
# Dataloader config
batch_size = 32

train_dataset, val_dataset = dataset_split(dataset, 0.8)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# for data in train_dataloader:
  # print(data['lyric'])

In [ ]:

'''
while True:
    model.train()
    total_loss = 0

    for batch in dataloader:
        lyrics = batch['lyric'].to(device)
        labels = batch['label'].to(device)
        attention_masks = batch['attention_mask'].to(device)

        optimizer.zero_grad()
        outputs = model(
            lyrics,
            token_type_ids=None,
            attention_mask=attention_masks,
            labels=labels
        )

        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

    avg_loss = total_loss / len(dataloader)
    print(f'Epoch {epoch + 1} - Loss: {avg_loss}', datetime.now().strftime("%H:%M:%S"))

    # Save Model

    model.save_pretrained('lyrics_model')
    tokenizer.save_pretrained('lyrics_model')

    epoch += 1
  '''

'\nwhile True:\n    model.train()\n    total_loss = 0\n\n    for batch in dataloader:\n        lyrics = batch[\'lyric\'].to(device)\n        labels = batch[\'label\'].to(device)\n        attention_masks = batch[\'attention_mask\'].to(device)\n\n        optimizer.zero_grad()\n        outputs = model(\n            lyrics,\n            token_type_ids=None,\n            attention_mask=attention_masks,\n            labels=labels\n        )\n\n        loss = outputs.loss\n        total_loss += loss.item()\n\n        loss.backward()\n        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)\n        optimizer.step()\n\n    avg_loss = total_loss / len(dataloader)\n    print(f\'Epoch {epoch + 1} - Loss: {avg_loss}\', datetime.now().strftime("%H:%M:%S"))\n\n    # Save Model\n\n    model.save_pretrained(\'lyrics_model\')\n    tokenizer.save_pretrained(\'lyrics_model\')\n\n    epoch += 1\n  '

In [ ]:
# tokenizer.model_max_length

In [ ]:
import logging

logger = logging.getLogger()  ###진행 과정에서 로깅 포인트가 발생할 경우 로깅을 하기 위한 코드입니다. 수정하실 필요 없습니다
logger.setLevel(logging.INFO)  ###진행 과정에서 로깅 포인트가 발생할 경우 로깅을 하기 위한 코드입니다. 수정하실 필요 없습니다

# Config
epochs = 3
warmup_ratio = 0.1
lr = 2e-5
grad_clip = 1.0
train_log_interval = 500 # train 이 100번 이루어질 때마다 logging# validation_interval = 1000
save_interval = 1000 ##save point는 1000번의 train


# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

# scheduler
data_len = len(train_dataloader)
num_train_steps = int(data_len / batch_size * epochs)
num_warmup_steps = int(num_train_steps * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_train_steps)


In [ ]:
# Train

def model_train(model, optimizer, scheduler, train_dataloader, device):
    model.to(device)  # 모델 학습을 설정된 device (CPU, cuda) 위에서 진행하도록 설정

    model.train() # 모델을 학습 모드로 전환
    loss_list_between_log_interval = []

    for epoch_id in range(epochs):

        for step_index, batch_data in tqdm(enumerate(train_dataloader), f"[TRAIN] Epoch:{epoch_id+1}", total=len(train_dataloader)):
                global_step = len(train_dataloader) * epoch_id + step_index + 1

                # Add a condition to break the loop if we've gone through all data points
                if step_index * batch_size >= len(dataset):
                  continue

                optimizer.zero_grad()
                lyrics = batch_data['lyric']
                labels = batch_data['label']
                attention_masks = batch_data['attention_mask']

                # 모델의 input들을 device(GPU)와 호환되는 tensor로 바꿔줍니다.
                lyrics = lyrics.to(device)
                labels = labels.to(device)
                attention_masks = attention_masks.to(device)
                # enocded_lyric = labels.to(deivce) # 수정

                model_outputs = model(
                    lyrics, token_type_ids=None, attention_mask=attention_masks, labels=labels
                    )

                loss = model_outputs.loss
                loss.backward()

                torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
                optimizer.step()
                scheduler.step()

                # for logging
                loss_list_between_log_interval.append(model_outputs.loss.item())

                #if global_step % train_log_interval == 0:

        mean_loss = np.mean(loss_list_between_log_interval)

        # 콘솔에 출력되는 부분
        logger.info(
            f"EP:{epoch_id} global_step:{global_step} "
            f"loss:{mean_loss:.4f} perplexity:{math.exp(mean_loss):.4f}"
        )

        loss_list_between_log_interval.clear()

        # if global_step % validation_interval == 0:
        # dev_loss = _validate(model, val_dataloader, device, logger, global_step)

        # 모델 저장 코드
        #state_dict = model.state_dict()
        # model_path = os.path.join('/content/drive/MyDrive/ML/my_AIKU/DeepIntoDeep/checkpoints', f"kobert_epoch_{epoch_id}.pth")
        #model_path = os.path.join('/content/drive/MyDrive/D2D/2023_SUMMER_AIKUTHON/checkpoints', f"kobert_epoch_{epoch_id}.pth")
        # logger.info(f"global_step: {global_step} model saved at {model_path}")
        #torch.save(state_dict, model_path)

    return model

In [ ]:
# 실제 훈련
model_v1 = model_train(model, optimizer, scheduler, train_dataloader, device)

[TRAIN] Epoch:1: 100%|██████████| 2750/2750 [31:04<00:00,  1.47it/s]
INFO:root:EP:0 global_step:2750 loss:0.2758 perplexity:1.3176
[TRAIN] Epoch:2: 100%|██████████| 2750/2750 [31:01<00:00,  1.48it/s]
INFO:root:EP:1 global_step:5500 loss:0.2342 perplexity:1.2639
[TRAIN] Epoch:3: 100%|██████████| 2750/2750 [31:01<00:00,  1.48it/s]
INFO:root:EP:2 global_step:8250 loss:0.2196 perplexity:1.2456


# Evaluation

In [ ]:
# inference

def model_inference(model, val_dataloader):
  predictions = []
  groundtruths = []

  model.eval()

  for batch_data in tqdm(val_dataloader):
    with torch.no_grad():
                lyrics = batch_data['lyric']
                labels = batch_data['label']
                attention_masks = batch_data['attention_mask']
                gt_labels=batch_data['gt_label']

                lyrics = lyrics.to(device)
                labels = labels.to(device)
                attention_masks = attention_masks.to(device)
                # enocded_lyric = labels.to(deivce) # 수정

                outputs = model(
                    lyrics, token_type_ids=None, attention_mask=attention_masks, labels=labels
                    )


                logits = outputs.logits
                #print(logits)
                # Predict Genre (숫자값)
                predicted_labels = torch.argmax(logits, dim=1)
                predictions.append(predicted_labels)
                groundtruths.append(gt_labels)
                '''
                int_to_genre = {0:'발라드',
                                1:'록/메탈',
                                2:'댄스',
                                4:'POP',
                                5:'R&B/Soul',
                                7: '랩/힙합',
                                16: '성인가요/트로트'}
                '''

  return predictions, groundtruths

In [ ]:
# 모델의 예측값

pred, gt = model_inference(model_v1, val_dataloader)
# pred = pred.tolist()
# print(pred)
# print(len(val_dataloader))

100%|██████████| 688/688 [02:27<00:00,  4.67it/s]


In [ ]:
output=torch.cat(pred)

In [ ]:
fin_pred = output.tolist()

In [ ]:
fin_pred

In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.8 MB/s eta 0:00:00


In [ ]:
gt=torch.cat(gt).tolist()

In [ ]:
gt

In [ ]:
import evaluate

accuracy_metric = evaluate.load("accuracy")
results = accuracy_metric.compute(references=gt, predictions=fin_pred)
print(results)

{'accuracy': 0.6432727272727272}


In [ ]:
'''
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)
'''

'\nimport numpy as np\n\ndef compute_metrics(eval_pred):\n    predictions, labels = eval_pred\n    predictions = np.argmax(predictions, axis=1)\n    return accuracy.compute(predictions=predictions, references=labels)\n'

# Test (Submission)

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/D2D/2023_SUMMER_AIKUTHON/test.csv', header=None, names=['label', 'lyric'])

In [ ]:
test_lyrics = test_df['lyric'][1:].tolist()

In [ ]:
cls_preprocess(test_lyrics)

In [ ]:
encoded_lyrics = tokenizer.batch_encode_plus(
    test_lyrics,
    add_special_tokens=True,
    padding='longest',
    truncation=True,
    max_length=512, # 수정
    return_attention_mask=True,
    return_tensors='pt'
)

In [ ]:
class LyricsDataset2(Dataset):
    def __init__(self, lyrics, attention_masks):
        self.lyrics = lyrics
        self.attention_masks = attention_masks
        self.num_classes = 7

    def __len__(self):
        return len(self.lyrics)

    def __getitem__(self, idx):
        return {
            'lyric': self.lyrics[idx],
            'attention_mask': self.attention_masks[idx],
        }


        num_classes = len(set(labels))


# Create dataset

dataset = LyricsDataset2(
    lyrics=encoded_lyrics['input_ids'],
    attention_masks=encoded_lyrics['attention_mask'],
)

In [ ]:
len(dataset)

10500

In [ ]:
test_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [ ]:
len(test_dataloader)

329

In [ ]:
def model_inferenc2(model, val_dataloader):
  predictions = []
 # groundtruths = []

  model.eval()

  for batch_data in tqdm(val_dataloader):
    with torch.no_grad():
                lyrics = batch_data['lyric']
  #              labels = batch_data['label']
                attention_masks = batch_data['attention_mask']
  #             gt_labels=batch_data['gt_label']

                lyrics = lyrics.to(device)
  #              labels = labels.to(device)
                attention_masks = attention_masks.to(device)
                # enocded_lyric = labels.to(deivce) # 수정

                outputs = model(
                    lyrics, token_type_ids=None, attention_mask=attention_masks
                    )


                logits = outputs.logits
                #print(logits)
                # Predict Genre (숫자값)
                predicted_labels = torch.argmax(logits, dim=1)
                predictions.append(predicted_labels)
  #              groundtruths.append(gt_labels)
                '''
                int_to_genre = {0:'발라드',
                                1:'록/메탈',
                                2:'댄스',
                                4:'POP',
                                5:'R&B/Soul',
                                7: '랩/힙합',
                                16: '성인가요/트로트'}
                '''

  return predictions

In [ ]:
predictions = model_inferenc2(model_v1, test_dataloader)

100%|██████████| 329/329 [01:10<00:00,  4.67it/s]


In [ ]:
len(predictions)

329

In [ ]:
predictions

[tensor([2, 3, 6, 0, 3, 0, 2, 0, 3, 1, 0, 3, 0, 0, 0, 3, 2, 3, 5, 0, 3, 6, 0, 1,
         3, 5, 5, 3, 6, 4, 2, 0], device='cuda:0'),
 tensor([2, 0, 2, 0, 6, 2, 2, 3, 6, 6, 3, 3, 4, 1, 0, 0, 1, 2, 3, 6, 0, 2, 3, 2,
         0, 3, 2, 3, 3, 5, 6, 0], device='cuda:0'),
 tensor([6, 3, 2, 0, 3, 0, 2, 6, 6, 0, 4, 3, 1, 2, 0, 0, 2, 0, 6, 3, 5, 5, 2, 0,
         0, 0, 1, 0, 3, 3, 1, 5], device='cuda:0'),
 tensor([6, 2, 3, 0, 0, 4, 1, 4, 2, 3, 0, 6, 0, 3, 0, 3, 3, 6, 0, 5, 2, 3, 1, 0,
         6, 5, 5, 1, 3, 4, 5, 2], device='cuda:0'),
 tensor([0, 3, 3, 4, 6, 2, 3, 6, 4, 2, 3, 0, 5, 1, 2, 0, 0, 2, 1, 6, 4, 0, 6, 1,
         3, 6, 3, 4, 5, 2, 0, 3], device='cuda:0'),
 tensor([2, 3, 2, 5, 6, 4, 6, 1, 4, 0, 1, 0, 2, 6, 1, 2, 0, 5, 0, 0, 5, 0, 1, 3,
         1, 0, 0, 1, 2, 4, 1, 0], device='cuda:0'),
 tensor([0, 4, 0, 0, 0, 3, 3, 6, 0, 0, 2, 0, 3, 0, 0, 1, 3, 2, 2, 0, 4, 2, 0, 5,
         5, 3, 6, 1, 0, 2, 5, 6], device='cuda:0'),
 tensor([6, 0, 5, 0, 4, 0, 0, 0, 0, 5, 0, 5, 6, 0, 6, 0, 5, 0, 0, 1, 

In [ ]:
predictions[-1]

tensor([5, 3, 5, 2], device='cuda:0')

In [ ]:
predictions2 = (torch.concat(predictions)).tolist()

In [ ]:
predictions2

[2,
 3,
 6,
 0,
 3,
 0,
 2,
 0,
 3,
 1,
 0,
 3,
 0,
 0,
 0,
 3,
 2,
 3,
 5,
 0,
 3,
 6,
 0,
 1,
 3,
 5,
 5,
 3,
 6,
 4,
 2,
 0,
 2,
 0,
 2,
 0,
 6,
 2,
 2,
 3,
 6,
 6,
 3,
 3,
 4,
 1,
 0,
 0,
 1,
 2,
 3,
 6,
 0,
 2,
 3,
 2,
 0,
 3,
 2,
 3,
 3,
 5,
 6,
 0,
 6,
 3,
 2,
 0,
 3,
 0,
 2,
 6,
 6,
 0,
 4,
 3,
 1,
 2,
 0,
 0,
 2,
 0,
 6,
 3,
 5,
 5,
 2,
 0,
 0,
 0,
 1,
 0,
 3,
 3,
 1,
 5,
 6,
 2,
 3,
 0,
 0,
 4,
 1,
 4,
 2,
 3,
 0,
 6,
 0,
 3,
 0,
 3,
 3,
 6,
 0,
 5,
 2,
 3,
 1,
 0,
 6,
 5,
 5,
 1,
 3,
 4,
 5,
 2,
 0,
 3,
 3,
 4,
 6,
 2,
 3,
 6,
 4,
 2,
 3,
 0,
 5,
 1,
 2,
 0,
 0,
 2,
 1,
 6,
 4,
 0,
 6,
 1,
 3,
 6,
 3,
 4,
 5,
 2,
 0,
 3,
 2,
 3,
 2,
 5,
 6,
 4,
 6,
 1,
 4,
 0,
 1,
 0,
 2,
 6,
 1,
 2,
 0,
 5,
 0,
 0,
 5,
 0,
 1,
 3,
 1,
 0,
 0,
 1,
 2,
 4,
 1,
 0,
 0,
 4,
 0,
 0,
 0,
 3,
 3,
 6,
 0,
 0,
 2,
 0,
 3,
 0,
 0,
 1,
 3,
 2,
 2,
 0,
 4,
 2,
 0,
 5,
 5,
 3,
 6,
 1,
 0,
 2,
 5,
 6,
 6,
 0,
 5,
 0,
 4,
 0,
 0,
 0,
 0,
 5,
 0,
 5,
 6,
 0,
 6,
 0,
 5,
 0,
 0,
 1,
 6,
 0,
 0,
 3,
 3,
 6,


In [ ]:
len(predictions2)

10500

In [ ]:
predictions3 = []
for i in tqdm(range(len(predictions2))):
    data = predictions2[i]
    if data == 3 : predictions3.append('4')
    elif data == 4 : predictions3.append('5')
    elif data == 5 : predictions3.append('7')
    elif data == 6 : predictions3.append('16')
    elif data == 0 : predictions3.append('0')
    elif data == 1 : predictions3.append('1')
    elif data == 2 : predictions3.append('2')
    else : predictions3.append(str(data))

100%|██████████| 10500/10500 [00:00<00:00, 1338729.73it/s]


In [ ]:
predictions3

In [ ]:
test_df2 = pd.read_csv('/content/drive/MyDrive/D2D/2023_SUMMER_AIKUTHON/test.csv')

In [ ]:
test_df = test_df2['Unnamed: 0']

In [ ]:
test_df = test_df.to_frame()

In [ ]:
len(test_df)

10500

In [ ]:
student_card = pd.DataFrame({'id':test_df2[['Unnamed: 0']],
                             'genre':predictions3})

ValueError: ignored

AttributeError: ignored

In [ ]:
df_fin = test_df

In [ ]:
# id
df_fin['id'] = test_df2[['Unnamed: 0']]

# genre
df_fin['genre'] = predictions3

#file
df_fin.to_csv('/content/drive/MyDrive/D2D/2023_SUMMER_AIKUTHON/result2.csv',sep=',', columns = ['id', 'genre'], index = False) # do not write index

In [ ]:
'''
# 훈련 함수 - Github version

model.train()
total_loss = 0

for batch in dataloader:
  lyrics = batch['lyric'].to(device)
  labels = batch['label'].to(device)
  attention_masks = batch['attention_mask'].to(device)

  optimizer.zero_grad()
  outputs = model(
            lyrics,
            token_type_ids=None,
            attention_mask=attention_masks,
            labels=labels
        )

  loss = outputs.loss
  total_loss += loss.item()

  loss.backward()
  torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
  optimizer.step()

  avg_loss = total_loss / len(dataloader)
  print(f'Epoch {epoch + 1} - Loss: {avg_loss}', datetime.now().strftime("%H:%M:%S"))
'''

# Train